# Project: Approximation of Option Premiums Using The Greeks
Fill me in

## Setup
Set up the computational environment by including the `Include.jl` file. The `Include.jl` file loads external packages, various functions we will use in the exercise, and custom types to model the components of our example problem.
* For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/) and the [VLQuantitativeFinancePackage.jl documentation](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl). 

In [3]:
include("Include.jl")

    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
   Resolving package versions...
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-134/module-3/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-134/module-3/Manifest.toml`
  Activating project at `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-134/module-3`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-134/module-3/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-134/module-3/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-134/module-3/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CH

## Prerequisites
Before we estimate how `call` and `put` option premiums change with market conditions using [the Greeks](https://en.wikipedia.org/wiki/en:Greeks_(finance)?variant=zh-tw), let's specify values for [the Greeks](https://en.wikipedia.org/wiki/en:Greeks_(finance)?variant=zh-tw) and some other parameters for our test `call` and `put` options on [NVIDIA Corporation](https://finance.yahoo.com/quote/NVDA/).

### Parameters and constants
Let's set a value for the step size `Δt,` i.e., `1 day` in units of years (assuming a `365-day` year), the risk-free rate at the time we gathered the data for the option contracts; we approximate this value using the [yield on the 10-year Treasury Note](https://ycharts.com/indicators/10_year_treasury_rate). We also set a value for the underlying share price `Sₒ` and the number of levels in the binomial tree model, `h`; we assume 6-levels per day.

In [221]:
Δt = (1/365);
risk_free_rate = 0.0431; # yield 10-year treasury note on 13-Jun-2024
Sₒ = 131.88; # after close on 14-Jun-2024 (close: 131.88)
h = 366; # 6-points per trading day
β = range(0.95,stop = 1.05, length=11) |> collect # peturbation vector

11-element Vector{Float64}:
 0.95
 0.96
 0.97
 0.98
 0.99
 1.0
 1.01
 1.02
 1.03
 1.04
 1.05

### Contract models
Next, we construct contract models for the options in this example. We begin by building an instance of the [MyAmericanCallContractModel type](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.MyAmericanCallContractModel) using the [custom build(...) method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.build-Tuple{Type{MyAmericanCallContractModel},%20NamedTuple}). The [build(...) method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.build-Tuple{Type{MyAmericanCallContractModel},%20NamedTuple}) takes two arguments: the type to build, and a [NamedTuple](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple) containing the strike price `K`, the `premium` paid for the option, the days to expiration `DTE` (units: years), the `sense = 1` (long), the number of contracts in the `copy` field and the implied volatility in the `IV` field.

We save the call option contract model in the `call_option_model` variable:

In [8]:
call_option_model = build(MyAmericanCallContractModel, (
    K = 136.0, 
    premium = 9.23, # mark price
    DTE = (61.0)*Δt,
    sense = 1,
    copy = 1,
    IV = 0.4848
));

The put contract, which is an instance of the [MyAmericanPutContractModel type](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.MyAmericanPutContractModel), is constructed in a similar way to the call option. We save the put option contract model in the `put_option_model` variable:

In [10]:
put_option_model = build(MyAmericanPutContractModel, (
    K = 128.0, 
    premium = 7.78, # mark price
    DTE = (61.0)*Δt,
    sense = 1,
    copy = 1,
    IV = 0.4787
));

### Calculate the Greek values
Fill me in

In [30]:
long_greek_dictionary = Dict{Union{MyAmericanPutContractModel,MyAmericanCallContractModel}, NamedTuple}();

Compute and store the Greeks for the `call` option

In [32]:
long_greek_dictionary[call_option_model] = (
    delta_value = delta(call_option_model, h = h,  T = call_option_model.DTE, 
        σ = call_option_model.IV, Sₒ = Sₒ, μ = risk_free_rate),
    theta_value = theta(call_option_model, h = h,  T = call_option_model.DTE, 
        σ = call_option_model.IV, Sₒ = Sₒ, μ = risk_free_rate),
    vega_value = vega(call_option_model, h = h,  T = call_option_model.DTE, 
        σ = call_option_model.IV, Sₒ = Sₒ, μ = risk_free_rate),
    rho_value = rho(call_option_model, h = h,  T = call_option_model.DTE, 
        σ = call_option_model.IV, Sₒ = Sₒ, μ = risk_free_rate),
    gamma_value = gamma(call_option_model, h = h,  T = call_option_model.DTE, 
        σ = call_option_model.IV, Sₒ = Sₒ, μ = risk_free_rate));

Compute and store the Greeks for the `put` option.

In [34]:
long_greek_dictionary[put_option_model] = (
    delta_value = delta(put_option_model, h = h,  T = put_option_model.DTE, 
        σ = put_option_model.IV, Sₒ = Sₒ, μ = risk_free_rate),
    theta_value = theta(put_option_model, h = h,  T = put_option_model.DTE, 
        σ = put_option_model.IV, Sₒ = Sₒ, μ = risk_free_rate),
    vega_value = vega(put_option_model, h = h,  T = put_option_model.DTE, 
        σ = put_option_model.IV, Sₒ = Sₒ, μ = risk_free_rate),
    rho_value = rho(put_option_model, h = h,  T = put_option_model.DTE, 
        σ = put_option_model.IV, Sₒ = Sₒ, μ = risk_free_rate),
    gamma_value = gamma(put_option_model, h = h,  T = put_option_model.DTE, 
        σ = put_option_model.IV, Sₒ = Sₒ, μ = risk_free_rate));

## Task 1: Compute changes in the premium for call options

In [223]:
call_greek_tuple = long_greek_dictionary[call_option_model];
call_greek_vector = [
    call_greek_tuple.delta_value,
    call_greek_tuple.gamma_value,
    call_greek_tuple.theta_value,
    call_greek_tuple.vega_value,
    call_greek_tuple.rho_value
]

5-element Vector{Float64}:
  0.4920000000000009
  0.022999999999999687
 -0.09199999999999875
  0.21600000000000108
  0.0940000000000012

In [225]:
long_call_perturbation_dictionary = Dict{Tuple{Float64,Float64}, Float64}();
tmp_array = Array{Float64,2}(undef,11,11);
IVₒ = call_option_model.IV;
for i ∈ eachindex(β)

    β_outer = β[i];
    S = Sₒ*(β_outer); # outer loop will be share price
    
    for j ∈ eachindex(β)
        β_inner = β[j];
        IV = (IVₒ)*β_inner; # inner loop will be implied volatility         
        δ_vector = [
            (β_outer - 1)*Sₒ, (1/2)*(S-Sₒ)^2, 0, (β_inner - 1)*IVₒ, 0
        ];
        old_premium = call_option_model.premium;
        long_call_perturbation_dictionary[(β_outer, β_inner)] = dot(call_greek_vector, δ_vector)/old_premium
        tmp_array[i,j] = dot(call_greek_vector, δ_vector)/old_premium
    end
end
long_call_perturbation_dictionary

Dict{Tuple{Float64, Float64}, Float64} with 121 entries:
  (1.02, 1.02) => 0.149491
  (1.05, 0.97) => 0.405324
  (0.96, 0.96) => -0.246974
  (1.04, 0.97) => 0.315523
  (0.98, 1.05) => -0.131361
  (0.97, 0.98) => -0.191618
  (1.05, 0.96) => 0.40521
  (1.04, 0.96) => 0.315409
  (0.98, 0.97) => -0.132268
  (0.99, 0.99) => -0.0682444
  (1.0, 0.95)  => -0.000567263
  (1.02, 1.03) => 0.149604
  (0.96, 0.99) => -0.246633
  (1.02, 1.01) => 0.149377
  (0.98, 0.96) => -0.132382
  (0.99, 1.0)  => -0.0681309
  (1.0, 1.04)  => 0.000453811
  (1.05, 0.99) => 0.40555
  (0.96, 1.0)  => -0.24652
  (0.97, 0.95) => -0.191958
  (1.04, 0.99) => 0.31575
  (1.02, 0.98) => 0.149037
  (1.0, 1.05)  => 0.000567263
  (1.05, 1.0)  => 0.405664
  (0.97, 1.04) => -0.190937
  ⋮            => ⋮

### Vis

In [143]:
max_element = maximum(tmp_array)
min_element = minimum(tmp_array)
rescaled_results_array = Array{Float64,2}(undef, 11,11)
for i ∈ eachindex(β)
    for j ∈ eachindex(β)
       rescaled_results_array[i,j] =  2*((tmp_array[i,j] - min_element)/(max_element - min_element)) - 1
    end
end

In [159]:
rescaled_results_array

11×11 Matrix{Float64}:
 -0.485026   -0.484786  -0.484547   …  -0.483111   -0.482872   -0.482633
 -0.74809    -0.747851  -0.747611      -0.746176   -0.745936   -0.745697
 -0.919748   -0.919509  -0.919269      -0.917834   -0.917594   -0.917355
 -1.0        -0.999761  -0.999521      -0.998086   -0.997846   -0.997607
 -0.988846   -0.988607  -0.988367      -0.986932   -0.986692   -0.986453
 -0.886286   -0.886046  -0.885807   …  -0.884371   -0.884132   -0.883893
 -0.692319   -0.69208   -0.691841      -0.690405   -0.690166   -0.689927
 -0.406947   -0.406708  -0.406468      -0.405033   -0.404793   -0.404554
 -0.0301683  -0.029929  -0.0296897     -0.0282541  -0.0280148  -0.0277755
  0.438016    0.438256   0.438495       0.439931    0.44017     0.440409
  0.997607    0.997846   0.998086   …   0.999521    0.999761    1.0

In [165]:
tmp_array

11×11 Matrix{Float64}:
  1.89687      1.89801      1.89914     …   1.90708      1.90822
  0.649572     0.650707     0.651841        0.659783     0.660918
 -0.164332    -0.163197    -0.162063       -0.154121    -0.152986
 -0.54484     -0.543706    -0.542571       -0.53463     -0.533495
 -0.491954    -0.49082     -0.489685       -0.481743    -0.480609
 -0.00567263  -0.00453811  -0.00340358  …   0.00453811   0.00567263
  0.914004     0.915138     0.916273        0.924215     0.925349
  2.26708      2.26821      2.26934         2.27729      2.27842
  4.05354      4.05468      4.05581         4.06375      4.06489
  6.2734       6.27454      6.27567         6.28361      6.28475
  8.92666      8.9278       8.92893     …   8.93687      8.93801

In [169]:
tmp_array[1,1]

1.8968713976163876

In [171]:
(β[1],β[1])

(0.5, 0.5)

## Task 2: Compute changes in the premium for put options

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. You should decide solely based on your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.